In [1]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
import cv2 
import os
import fnmatch
from sklearn import svm
from sklearn.metrics import accuracy_score

In [2]:
train_path = "Demo_dataset"
train_chinese = os.listdir(train_path)
df_train = pd.DataFrame(columns=["character", "filename", "label"])
df_test = pd.DataFrame(columns=["character", "filename", "label"])
len(train_chinese)

8

In [3]:
def image_preprocessing(df_train):
    array = np.zeros((df_train.shape[0], 128, 128, 3))
    for i in range(df_train.shape[0]):
        if df_train.iloc[i]['filename'] != ".DS_Store":
            image_path = train_path + '/' + df_train.iloc[i]['character'] + '/' + df_train.iloc[i]['filename']
            # get grayscale image
            # print(image_path)
            image = cv2.imread(image_path)
            # remove noise
            out = cv2.medianBlur(image, 5)
            # normalize the image
            resultimage = np.zeros(out.shape)
            normalizedimage = cv2.normalize(out, resultimage, 0, 255, cv2.NORM_MINMAX)
            # resize the image 
            resizeimage = cv2.resize(normalizedimage, (128,128))

            blue, green, red = cv2.split(resizeimage) 
            # apply principal component analysis
            pca = PCA(50)
            red_transformed = pca.fit_transform(red)
            red_inverted = pca.inverse_transform(red_transformed)

            #Applying to Green channel and then applying inverse transform to transformed array.
            green_transformed = pca.fit_transform(green)
            green_inverted = pca.inverse_transform(green_transformed)

            #Applying to Blue channel and then applying inverse transform to transformed array.
            blue_transformed = pca.fit_transform(blue)
            blue_inverted = pca.inverse_transform(blue_transformed)

            img_compressed = (np.dstack((red_inverted, green_inverted, blue_inverted))).astype(np.uint8)
            array[i] = img_compressed
    return array

In [4]:
k = 0
for i in range(len(train_chinese)):
    if i != ".DS_Store":
        l = os.listdir(train_path + '/' + train_chinese[i])
        l = fnmatch.filter(l, '*.png')
        l = np.array(l)
        kf = KFold(n_splits = 5, random_state = None, shuffle = False)
        if len(l) >= 200:
            l = l[:200]
            for j, (train_index, test_index) in enumerate(kf.split(l)):
                if len(l) >= 200:
                    l = l[:200]
                    label_train = [k] * len(l[train_index])
                    label_test = [k] * len(l[test_index])
                    temp1 = pd.DataFrame({
                        "character": train_chinese[i],
                        "filename": l[train_index],
                        "label": label_train
                    })
                    temp2 = pd.DataFrame({
                        "character": train_chinese[i],
                        "filename": l[test_index],
                        "label": label_test
                    })
                    #print(temp)
                    df_train = pd.concat([df_train, temp1])  
                    df_test = pd.concat([df_test, temp2])
        k += 1
                

In [5]:
train_array = image_preprocessing(df_train)
test_array = image_preprocessing(df_test)
#print(train_array.shape)
#print(test_array.shape)
X_train = np.array(train_array).reshape(len(train_array),-1)
X_train = X_train / 255.0
y_train = df_train["label"].astype('int')

X_test = np.array(test_array).reshape(len(test_array),-1)
X_test = X_test / 255.0
y_test = df_test["label"].astype('int')

In [6]:
train_X = np.zeros((5, X_train.shape[0], 49152))
train_y = np.zeros((5, y_train.shape[0]))
test_X = np.zeros((5, X_test.shape[0], 49152))
test_y = np.zeros((5, y_test.shape[0]))

In [7]:
for i in range(8):
    x1 = X_train[800 * i : 800 * (i + 1)]
    y1 = y_train[800 * i : 800 * (i + 1)]
    x2 = X_test[200 * i : 200  * (i + 1)]
    y2 = y_test[200 * i : 200 * (i + 1)]
    for j in range(5):
        #print(i, j)
        #print(160 * i, 160 * (i+1))
        train_X[j][160 * i  : 160 * (i+1)]  = x1[160 * j : 160 * (j + 1)]
        train_y[j][160 * i  : 160 * (i+1)]  = y1[160 * j : 160 * (j + 1)]
        test_X[j][40 * i : 40 * (i + 1)] = x2[40 * j : 40  * (j + 1)]
        test_y[j][40 * i : 40 * (i + 1)] = y2[40 * j : 40  * (j + 1)]
        
    

In [8]:
accuracy = []
for i in range(5):
    svc=svm.SVC(kernel = 'linear', C = 0.1)
    svc.fit(train_X[i], train_y[i])
    y2 = svc.predict(test_X[i])
    accuracy.append(accuracy_score(test_y[i],y2))

In [9]:
final_accuracy = np.sum(accuracy) / 5
print("Accuracy for K-fold SVM Model is: ", final_accuracy)

Accuracy for K-fold SVM Model is:  0.9768749999999999


In [12]:
accuracy2 = []
for i in range(5):
    svc=svm.SVC(kernel = 'rbf', C = 0.1)
    svc.fit(train_X[i], train_y[i])
    y2 = svc.predict(test_X[i])
    accuracy2.append(accuracy_score(test_y[i],y2))

In [13]:
final_accuracy2 = np.sum(accuracy2) / 5
print("Accuracy for K-fold SVM Model is: ", final_accuracy2)

Accuracy for K-fold SVM Model is:  0.9581250000000001
